## Submission Notebook Template 

<h3> <font color='red'> WARNING : </font>  </h3>

<font color='red'> No matter which approach you've chosen, you need to re-install any custom packages you had to install to make your code work ! </font>

<b> Install your packages below: </b>

In [ ]:
!pip install git+https://github.com/Total-RD/pymgrid/
## Other packages 

In the section below, you must run your methodology for solving the problem from start to finish :

In [ ]:
import pickle

with open('data/building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)

with open('data/building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    
with open('data/building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)

buildings = [building_1, building_2, building_3]

<h2> Evaluation for Rule Based Approaches </h2>

If you're using Rule Based Algorithms, here is what your submitted code should feature no matter what 

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
import json # Necessary to export your results
## Other packages 

<b> 2) Implementation of the rules that generate control dictionaries </b>

In [ ]:
"""
EXAMPLE FOR A NAIVE RULE BASED STRATEGY
"""
def naive_rule_based_strategy(building):
    building_data = building.get_updated_values()

    total_building_cost = 0

    while building.done == False:
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']

        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))

        if load - pv >=  0:

            control_dict = {'battery_charge': 0,
                                'battery_discharge': p_disc,
                                'grid_import': max(0, load-pv-p_disc),
                                'grid_export':0,
                                'pv_consummed': min(pv, load),
                           }


        if load - pv <  0:

            control_dict = {'battery_charge': p_char,
                                'battery_discharge': 0,
                                'grid_import': 0,
                                'grid_export': max(0,pv-load-p_char),#abs(min(load-pv,0)),
                                'pv_consummed': min(pv, load+p_char),
                           }

        building_data = building.run(control_dict)
        total_building_cost += building.get_cost()
    
    return total_building_cost

<b> 3) Run of the rules on the Test environment </b>

In [ ]:
"""
Rule based methods have no "training" as such, this means Training CPU Time will always be 0 and only
Test CPU Time will represent frugality
"""

eval_start = time.process_time()

total_building_costs = []

for building in buildings:

    building.reset(testing = True)

    total_building_cost = naive_rule_based_strategy(building)
    total_building_costs.append(total_building_cost)

eval_end = time.process_time()



In [ ]:
total_cost_building_1 = total_building_costs[0]
total_cost_building_2 = total_building_costs[1]
total_cost_building_3 = total_building_costs[2]

In [ ]:
frugality = eval_end - eval_start

<b> 4) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost_building_1,
    "building_2_performance" : total_cost_building_2,
    "building_3_performance" : total_cost_building_3,
    "frugality" : frugality,
}
print(final_results)

In [ ]:
team_name = ## Enter Team name here

with open(team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)

<h2> Evaluation for "Simple" Reinforcement Learning based approaches <h2>

 <font color='red'> <b> Be careful, the rewards returned by the Gym environment are negative ! Don't forget to multiply by -1 the total reward as the Profitability you will need to submit needs to be positive ! </b> </font>

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
import json # Necessary to export your results
import DiscreteEnvironment as DiscreteEnvironment # Imposed Discrete Environment
import DiscreteEnvironment_modified as DiscreteEnvironment_modified # Imposed Discrete Environment
import numpy as np

## Other packages

<b> 2) Agent & Environment Setup before your training </b>

In [ ]:
building_environments = [
    DiscreteEnvironment_modified.Environment(env_config={'building':buildings[i]}) for i in range(3)
]
"""
Agent, potential Q-Table & other necessary setup code here 
"""


**Functions for Q-learning algo**  
We used the functions developped in the following repo : https://github.com/Total-RD/pymgrid/blob/master/notebooks/A%20Q-Learning%20Example%20with%20PymGrid.ipynb, and modified them when needed.

In [ ]:
def init_qtable(env, nb_action, building):
    """
    Initialize the Q-table of the Q-learning.
    The state is an intersection between net load possibilities (from -(pv production) to load),
    and the state of charge of the battery (from soc_min and soc_max).

    -- Input :
        env : Enivronment object
        nb_action : Number of action can be taken by each state during the Q-learning [Integer]

    -- Ouput :
        Q : A dict of state containing the value of weight of each action for each state
            (Dict of Dict)
    """

    state = []
    Q = {}

    # --- Defining state possibilities ---------------------------
    for i in range(-int(env.parameters["PV_rated_power"] - 1), int(env.parameters["load"] + 2)):
        for j in np.arange(round(env.battery.soc_min, 1), round(env.battery.soc_max + 0.1, 1), 0.1):
            state.append((i, round(j, 1)))

    # --- Initialize Q(s, a) at zero ----------------------------
    for s in state:
        Q[s] = {}
        for a in range(nb_action):
            if building==3:
                if a==4:
                    Q[s][a] = -50
                elif a==6:
                    Q[s][a] = -30
                else:
                    Q[s][a] = 0
            else:
                Q[s][a] = 0

    return Q

def epsilon_decreasing_greedy(action, epsilon, nb_action):
    """
    Adding random aleas for the choice of actions

    -- Input :
        action : integer representing the action taken [Integer]
        epsilon : share of aleas to consider (biggest espsilon is and biggest part of
                  aleas is taken [Float]
        nb_action : Number of action can be taken by each state during the Q-learning [Integer]

    -- Output :
        action : integer representing the action taken [Integer]
        randomm : binary value to consider if a aleas has been taken for action choice [Integer]
    """
    p = np.random.random()

    if p < (1 - epsilon):
        randomm = 0
        return action, randomm

    else:
        randomm = 1
        return np.random.choice(nb_action), randomm
    
def max_dict(d):
    """
    Returning the tuple (action, val) maximizing the reward in the Q-table depending of a state.
    Reward correspond to the amount paid to answer the consumption (count negatively)
    => Maximizing a negative number = Minimizing its absolute value

    -- Input :
        d : dict of action and reward associate of a state [Dict]

    -- Output :
        max_key : action corresponding to the maximal reward [Integer]
        max_value : value of the maximal reward [Integer]
    """
    max_key = None
    max_val = float("-inf")

    for k, v in d.items():
        if v > max_val:
            max_val = v
            max_key = k

    return max_key, max_val

def update_epsilon(epsilon):
    """
    Update epsilon value (share of aleas in the choice of an action) to minimize it through iteration.

    -- Input :
        epsilon : share of aleas to consider (biggest espsilon is and biggest part of
                  aleas is taken [Float]

    -- Output :
        epsilon (updated) : share of aleas to consider (biggest espsilon is and biggest part of
                            aleas is taken [Float]
    """
    epsilon = epsilon - epsilon * 0.02

    if epsilon < 0.1:
        epsilon = 0.1

    return epsilon

def change_name_action(idx, building):
    """
    Print function
    """
    if building==3:
        if idx == 0:
            action_name = "PV + Charge + Export"
        elif idx == 5:
            action_name = "PV + Discharge + Import"
        elif idx == 2:
            action_name = "Import"
        elif idx == 3:
            action_name = "Full Export"
        elif idx == 4:
            action_name = "Genset"
        elif idx == 1:
            action_name = "Export/Import"
        elif idx == 6:
            action_name = "Genset"
    else:
        if idx == 0:
            action_name = "PV + Charge + Export"
        elif idx == 1:
            action_name = "PV + Discharge + Import"
        elif idx == 2:
            action_name = "Import"
        elif idx == 3:
            action_name = "Full Export"
        elif idx == 4:
            action_name = "Export/Import"
    
    return action_name

def print_welcome(idx):
    """
    Print function
    """
    if idx == 0:
        print("------------------------------------")
        print("|        WELCOME TO PYMGRID        |")
        print("------------------------------------")
    elif idx == 1:

        print("t -     STATE  -  ACTION - COST")
        print("================================")

In [ ]:
def training_Q_Learning_DE(env, nb_action, building, horizon):

    # --- Defining parameters ----------------------------------
    Q = init_qtable(env.mg, nb_action, building)
    nb_state = len(Q)
    nb_episode = 15
    alpha = 0.1    #  Learning rate
    epsilon = 0.1  #  Aleas
    gamma = 0.99
    record_cost = []
    t0 = time.time()
    t = t0
    print_training = "Training Progressing .   "
    print_welcome(0)
    print("\n")

    for e in range(nb_episode + 1):

        # --- Initialize episode variables --------------------------
        episode_cost = 0
        env.reset()
        net_load = round(env.mg.load - env.mg.pv)
        soc = round(env.mg.battery.soc, 1)
        s = (net_load, soc)  # First state
        a = max_dict(Q[s])[0]  # First action
        a, randomm = epsilon_decreasing_greedy(a, epsilon, nb_action)  # Adding aleas in the first action

        # --- Q-learning accros horizon ------------------------------
        for i in range(horizon):

            # Run action choosen precedently
            status, reward, done, info = env.step(a)
            
            # Compute cost with the previous actions
            r = reward
            episode_cost = env.get_cost()

            # Update variables depending on the precedent action
            net_load = round(env.mg.load - env.mg.pv)
            soc = round(env.mg.battery.soc, 1)
            s_ = (net_load, soc)
            a_ = max_dict(Q[s_])[0]

            if i == horizon - 1:
                Q[s][a] += alpha * (r - Q[s][a])

            # Update reward depending on the action choosen
            else:
                old_Q = Q[s][a]  # Previous reward
                target = (r + gamma * Q[s_][a_])  # Target = reward of the previous action + expectation of reward of the last action
                td_error = target - Q[s][a]  # Difference of cost between two episode
                Q[s][a] = (1 - alpha) * Q[s][a] + alpha * td_error  # Update weight in the Q-table with the reward of the last action
            s, a = s_, a_
        epsilon = update_epsilon(epsilon)

    return Q

def testing_Q_Learning_DE(env, Q, horizon, building):

    # --- Initialize variables --------------------------
    env.reset()
    net_load = round(env.mg.load - env.mg.pv)
    soc = round(env.mg.battery.soc, 1)
    s = (net_load, soc)
    a = max_dict(Q[s])[0]
    total_cost = 0
    print_welcome(1)

    # --- Q-learning accros horizon ----------------------
    for i in range(horizon):

        # Run action choosen precedently
        action_name = change_name_action(a, building)        
        status, reward, done, info = env.step(a)
        cost = - reward
        total_cost = env.get_cost()

        # Print function
        #if i % 500 == 0 or i == horizon - 1:
        #print(i, " -", (int(net_load), soc), action_name, round(total_cost, 1), "€")

        #  Update variables depending on the last action
        net_load = round(env.mg.load - env.mg.pv)
        soc = round(env.mg.battery.soc, 1)

        #  Defining the next state and action corresponding
        s_ = (net_load, soc)
        a_ = max_dict(Q[s_])[0]
        s, a = s_, a_
        
    return total_cost

<b> 3) Training of the agent </b>

In [ ]:

train_start = time.process_time()

"""
Training code
"""
Q0_DE = training_Q_Learning_DE(building_environments[0], 5, 1, 8757)
Q1_DE = training_Q_Learning_DE(building_environments[1], 5, 2, 8757)
Q2_DE = training_Q_Learning_DE(building_environments[2], 7, 3, 8757)

train_end = time.process_time()



In [ ]:
train_frugality = train_end - train_start
print(train_frugality)

<b> 4) Test of the agent </b>

In [ ]:
test_start = time.process_time()
total_cost = [0,0,0]

building_environments[0].reset(testing=True)
total_cost[0] = testing_Q_Learning_DE(building_environments[0], Q0_DE, 8757, 1)

building_environments[1].reset(testing=True)
total_cost[1] = testing_Q_Learning_DE(building_environments[1], Q1_DE, 8757, 2)

building_environments[2].reset(testing=True)
total_cost[2] = testing_Q_Learning_DE(building_environments[2], Q2_DE, 8757, 3)
    
test_end = time.process_time()

In [ ]:
"""
Below is an example for a Random Agent 

Note :
* To make your work as reproductible as possible, have a full-greedy approach (no exploration) on the test buildings
* If your algorithm has some unavoidable randomness, consider running it for many loops and return a
  mean profitability and mean frugality
  
"""

test_start = time.process_time()
total_cost = [0,0,0]

for i,building_env in enumerate(building_environments):
    
    obs = building_env.reset(testing=True)
    done = False
    while not done:
        action = np.random.randint(building_env.action_space.n)
        obs, reward, done, info = building_env.step(action)
        total_cost[i]+=reward

test_end = time.process_time()

In [ ]:
test_frugality = test_end - test_start
print(test_frugality)

<b> 5) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost[0],
    "building_2_performance" : total_cost[1],
    "building_3_performance" : total_cost[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)

In [ ]:
team_name = 'team35'

with open('results/' + team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)

<h2> Evaluation for Deep Reinforcement Learning based approaches <h2>

 <font color='red'> <b> Be careful, the rewards returned by the Gym environment are negative ! Don't forget to multiply by -1 the total reward as the Profitability you will need to submit needs to be positive ! </b> </font>

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv # Imposed Environment
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

## Import your favourite Deep Learning library for RL and other packages here

<b> 2) Agent & Environment Setup before your training </b>

In [ ]:
"""
Below is an environment initialization without a Deep RL library, the code can vary depending on which library you 
use
"""
building_environments = [MicroGridEnv(env_config={'microgrid':buildings[i]}) for i in range(3)]


<b> 3) Training of the agent </b>

In [ ]:
class DeepQL(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, nb_actions):
        super(DeepQL, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.nb_actions = nb_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.nb_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        
        if torch.cuda.is_available():       
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.to(self.device)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        
        return actions
        

In [ ]:
class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, nb_actions,
                max_mem_size = 100000, eps_end = 0.01, eps_dec = 5e-4):
        
        self.gamma = gamma
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(nb_actions)]
        self.mem_size = max_mem_size
        self.mem_counter = 0
        
        self.Q_eval = DeepQL(self.lr, nb_actions=nb_actions, input_dims=input_dims,
                            fc1_dims=64, fc2_dims=64)
        
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype= np.bool)
    
    def store_transition(self, state, action ,reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.new_state_memory[index] = state_
        self.terminal_memory[index] = done
        
        self.mem_counter += 1
    
    def choose_action (self, observation):
        if np.random.random() > self.epsilon:
            state = torch.FloatTensor([observation]).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            action = torch.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        
        return action
    
    def train(self):
        if self.mem_counter < self.batch_size:
            return
        
        self.Q_eval.optimizer.zero_grad()
        
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem,self.batch_size, replace=False)
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        state_batch = torch.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = torch.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = torch.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = torch.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        
        action_batch = self.action_memory[batch]
        
        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0
        
        q_target = reward_batch + self.gamma * torch.max(q_next,dim=1)[0]
        
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
    

In [ ]:
agent = Agent(gamma=0.99, epsilon=1.0, batch_size=128, nb_actions=5,
             eps_end=0.01, input_dims=[10], lr= 0.0001)

In [ ]:
train_start = time.process_time()

nb_episode = 15

for idx, env in enumerate(building_environments):
    print(f"building {idx}")
    for i in range(nb_episode):
        score = 0
        done = False
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score -= reward
            agent.store_transition(observation, action, reward, observation_, done)
            agent.train()
            observation = observation_
        print(f"- episode : {i} | score : {score}")
train_end = time.process_time()
train_frugality = train_end - train_start

In [ ]:
print(train_frugality)

<b> 4) Test of the agent </b>

In [ ]:
"""
Below is an example for a Random Agent 

Note :
* To make your work as reproductible as possible, have a full-greedy approach (no exploration) on the test buildings
* If your algorithm has some unavoidable randomness, consider running it for many loops and return a
mean profitability and mean frugality

"""

test_start = time.process_time()

test_start = time.process_time()
total_cost = [0,0,0]

for i,building_env in enumerate(building_environments):

    obs = building_env.reset(testing=True)
    done = False
    while not done:
        action = agent.choose_action(obs)
        obs, reward, done, info = building_env.step(action)
        total_cost[i]-=reward

test_end = time.process_time()

test_frugality = test_end - test_start

<b> 5) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost[0],
    "building_2_performance" : total_cost[1],
    "building_3_performance" : total_cost[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)

In [ ]:
import json
team_name = 'team35'

with open('results/' + team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)

<h2> For Optimisation based approaches <h2>

As explained in the README and the Getting Started Notebook, the evaluation depends heavily on how you formulate the problem, no template can be given for this familly of methods 